# Sales of summer clothes in E-commerce Wish
**Possible questions to answer from task.**

1. How about trying to validate the established idea of human sensitiveness to price drops ?
2. You may look for top categories of products so that you know what sells best
3. Do bad products sell ? How about the relationship between the quality of a product (ratings) and its success 4 Does the price factor into this ?
5. Do seller's fame factor into top products ?
6. Do the number of tags (making a product more discoverable) factor into the success of a product ?

# **Work is in progress**

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
%matplotlib inline

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
raw_data = pd.read_csv('/kaggle/input/summer-products-and-sales-in-ecommerce-wish/summer-products-with-rating-and-performance_2020-08.csv')#
raw_data.head()

In [ ]:
data=raw_data.copy()

**Creating a list of unnecessary columns:-**

In [ ]:
drop_col=['merchant_title','merchant_name','merchant_info_subtitle','merchant_rating_count','merchant_rating','merchant_has_profile_picture',
         'merchant_profile_picture','product_url','product_picture','crawl_month','title','title_orig']

**Creating a function to visualize drop-col :-**

In [ ]:
def graphs(column,data,column_to_exclude):
    if column not in column_to_exclude:
        if data[column].dtype not in['int64','float64']:
            f,ax=plt.subplots(1,1,figsize=(7,7))
            sns.countplot(x=column,data = data)
            plt.xticks(rotation=90)
            plt.suptitle(column)
            plt.show()
        else:
            g=sns.FacetGrid(data,margin_titles=True,aspect=4,height=4)
            g.map(plt.hist,column,bins=100)
            plt.show()
        plt.show()
        
#for column in raw_data.columns:
    graphs(column,raw_data,drop_col);

In [ ]:
data=data.drop(data[drop_col],axis=1)

In [ ]:
data.head()

In [ ]:
data=data.drop('urgency_text',axis=1)
data=data.drop(['currency_buyer','merchant_id','theme'],axis=1)

In [ ]:
data=data.drop(['shipping_option_name','shipping_is_express'],axis=1)
data['origin_country']=data['origin_country'].fillna('CN')


In [ ]:
data.isna().sum()

In [ ]:
data['has_urgency_banner']=data['has_urgency_banner'].fillna(data['has_urgency_banner'].mean())

In [ ]:
for col in data.columns:
    print(col,':',len(data[col].unique()),'label')

In [ ]:
data['product_variation_size_id'].value_counts()

In [ ]:
data['product_variation_size_id']=data['product_variation_size_id'].fillna('S')
data['product_color'].value_counts()

In [ ]:
data['product_color']=data['product_color'].fillna('black')

In [ ]:
plt.figure(figsize=(6,6))
sns.boxenplot(data['rating_five_count']);

**Checking the Rating:-**

In [ ]:
data[data['rating_five_count'].isna()==True][['rating','rating_count','rating_five_count','rating_four_count',
                                              'rating_three_count','rating_two_count','rating_one_count'
                                             ]].head(12)

**Creating a function which will identify the successful products according to their units_sold:-**

In [ ]:
def is_successful(col):
    if col>1000:
        return 1
    else:
        return 0
data['success']=data['units_sold'].apply(is_successful)
print('percent of successfull product is:',data['success'].value_counts()[1]/len(data['success'])*100)
sns.countplot(data=data,x='success');
plt.show()

CHECKING EFFECT OF PRICE V/S RETAIL_PRICE:

In [ ]:
print('///////////////////////////////////////////')
print('overall status:')
print('For price:',data['price'].mean())
print('Retail_price:',data['retail_price'].mean())
print('---------------------------------------')
print('Status for Successful product:')
print('For price:',data[data['success']==1]['price'].mean())
print('For retail_price:',data[data['success']==1]['retail_price'].mean())
print('////////////////////////////////////////////////////')
print('NO CONCLUSION')

In [ ]:
data['rating_five_count']=data['rating_five_count'].fillna(0)
data['rating_four_count']=data['rating_four_count'].fillna(0)
data['rating_three_count']=data['rating_three_count'].fillna(0)
data['rating_two_count']=data['rating_two_count'].fillna(0)
data['rating_one_count']=data['rating_one_count'].fillna(0)

In [ ]:
sns.boxenplot(data['countries_shipped_to'],data['origin_country']);

**Checking the effect of Ratings on success:-**

In [ ]:
print('Mean of Success by Ratings:-')
print('for five=',data[data['success']==1]['rating_five_count'].mean())
print('for four=',data[data['success']==1]['rating_four_count'].mean())
print('for three=',data[data['success']==1]['rating_three_count'].mean())
print('for two=',data[data['success']==1]['rating_two_count'].mean())
print('for one=',data[data['success']==1]['rating_one_count'].mean())

**** checking for unsuccessful:-****

In [ ]:
print('Mean of unsuccess by ratings:-')
print('for five=',data[data['success']==0]['rating_five_count'].mean())
print('for four=',data[data['success']==0]['rating_four_count'].mean())
print('for three=',data[data['success']==0]['rating_three_count'].mean())
print('for two=',data[data['success']==0]['rating_two_count'].mean())
print('for one=',data[data['success']==0]['rating_one_count'].mean())

**Now ,Visualizing the Graphs of units_sold and Ratings v/s success:-**

In [ ]:
fig,((ax1,ax2))=plt.subplots(1,2,figsize=(6,6),sharey=True)
ax1=sns.boxplot(data['success'],data['rating_five_count'])


ax2=sns.barplot(data['success'],data['units_sold'])
fig.suptitle('BEST SUCCESS AND FALIURE GRAPH(UNITS_SOLD &RATING)');


According to above graph there is not any unsuccessful product was sold

In [ ]:
#Creating a column of Top Products:-
data['top_product']=data[data['success']==1]['product_id']


**Checking the Effect of merchants'Fame and success:-**

In [ ]:
x=pd.DataFrame(raw_data['merchant_rating'],data['success'])
print('the Effect of merchant_rating v/s success is:-')

x.groupby(['success']).mean()


 ****So,there is not enough significant effect of merchant's fame and success of product

In [ ]:
sns.distplot(x);

**Checking for relationship of success and tags:-**

In [ ]:
y=pd.DataFrame(data['tags'],data['success'])
z=y.groupby(['success'])
z.describe(include='all')